# Preparing Documents

In [47]:
#dataset test 1
doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father."
doc2 = "My father spends a lot of time driving my sister around to dance practice."
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better."
doc5 = "Health experts say that Sugar is not good for your lifestyle."
doc_test1="sugar is bad"
doc_test2="sugar is good"
#doc_complete = [doc1, doc2]
doc_complete = [doc1, doc2, doc3, doc4, doc5]
#doc_complete = [doc_test1, doc_test2]

In [78]:
print '\n topi'.join(doc_complete)

Sugar is bad to consume. My sister likes to have sugar, but not my father.
 topiMy father spends a lot of time driving my sister around to dance practice.
 topiDoctors suggest that driving may cause increased stress and blood pressure.
 topiSometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better.
 topiHealth experts say that Sugar is not good for your lifestyle.


In [46]:
# dataset test 2
import pandas as pd
xl = pd.ExcelFile("data.xlsx")
df = xl.parse("data")
df.columns = ['sujet', 'doc']
doc_complete = df['doc']
print(documents)

0    Basketball is a non-contact team sport played ...
1    Football is a family of team sports that invol...
2    Association football, more commonly known as f...
3    American football, referred to as football in ...
4    A football team is the collective name given t...
5    Women's basketball is one of the few[citation ...
6    The five basketball positions normally employe...
7    The history of basketball is traced back to a ...
Name: doc, dtype: object


# Cleaning and Preprocessing

In [48]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string
tokenizer = RegexpTokenizer(r'\w+')
stop = set(stopwords.words('english'))

exclude = set(string.punctuation) 
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
lemma = WordNetLemmatizer()
def clean(doc):
    # clean and tokenize document string
    raw = doc.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in stop]
    
    # remove punc_free from tokens
    punc_free = [ch for ch in stopped_tokens if ch not in exclude]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in punc_free]
    
    # lemm tokens
    normalized = [lemma.lemmatize(word) for word in punc_free]
   
    # add tokens to list
    return normalized
 
doc_clean = [clean(doc) for doc in doc_complete] 
#print(doc_clean)
#print(doc_clean1)

# Preparing Document-Term Matrix

In [49]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

#print dic to ckeck
#for k, v in dictionary.iteritems():
    #print k, v

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

for i,doc in enumerate(doc_term_matrix):
    print "============ doc %s =======================" %(i)
    for d in doc:
        print "%s : %s"% (dictionary[d[0]], d[1])

============ doc 0 =======================
sister : 1
consume : 1
father : 1
sugar : 2
bad : 1
like : 1
============ doc 1 =======================
sister : 1
father : 1
around : 1
driving : 1
dance : 1
practice : 1
lot : 1
time : 1
spends : 1
============ doc 2 =======================
driving : 1
stress : 1
increased : 1
doctor : 1
may : 1
suggest : 1
pressure : 1
blood : 1
cause : 1
============ doc 3 =======================
sister : 1
father : 1
pressure : 1
school : 1
never : 1
perform : 1
seems : 1
sometimes : 1
well : 1
drive : 1
better : 1
feel : 1
============ doc 4 =======================
sugar : 1
health : 1
say : 1
good : 1
lifestyle : 1
expert : 1


In [50]:
for i in doc_term_matrix:
    print i
print "================="
print doc_term_matrix[0]


[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1)]
[(0, 1), (2, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]
[(7, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]
[(0, 1), (2, 1), (18, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]
[(3, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)]
[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1)]


# Running LDA Model

In [51]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
nb_topics=5
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=nb_topics, id2word = dictionary, passes=50)
print(ldamodel)

LdaModel(num_terms=35, num_topics=5, decay=0.5, chunksize=2000)


# Results

In [52]:
# words by topic
print(ldamodel.print_topics(num_topics=nb_topics, num_words=3))


[(0, u'0.075*"driving" + 0.075*"sister" + 0.075*"father"'), (1, u'0.029*"sugar" + 0.029*"driving" + 0.029*"sister"'), (2, u'0.029*"sugar" + 0.029*"driving" + 0.029*"sister"'), (3, u'0.063*"pressure" + 0.063*"sister" + 0.063*"father"'), (4, u'0.092*"sugar" + 0.092*"health" + 0.092*"say"')]


In [53]:
#parametre : doc par term
#resut: topic with score by doc id
print(ldamodel.get_document_topics(doc_term_matrix[1]))

[(0, 0.91982559354103033), (1, 0.020008397003264022), (2, 0.020008397003294244), (3, 0.020153248387034903), (4, 0.020004364065376649)]
